In [1]:
import numpy as np
import sys
import itertools as itertools
import time
sys.path.append("..")
sys.path.append("../..")

import matplotlib.pyplot as plt
from python.Heisenberg import *
from python.DMRG import DMRG
from python.Zippers import MPO_to_Hamiltonian, contract_MPS
from python.Canonical_Form import get_Neumann_entropy_from_left_isometry
from python.Backend import Backend
from python.Decomposition import EIGH

In [2]:
device = "numpy"
# device = "torch"

bk = Backend(device)

NSweep = 20
Krylov_bases = 5  #* 5 is usually enough
Lanczos_cutoff = 1e-2
two_site = True
verbose = True

In [3]:
import numpy.typing as npt
import pickle
from dataclasses import dataclass, field, asdict, replace
from copy import deepcopy
from typing import Any, Optional
from pathlib import Path


@dataclass
class Params:
    L: int
    XY: float
    ZZ: float
    h: float
    NKeep: int
    NSweep: int
    diag: bool
    Krylov: int
    cutoff: float
    two: bool
    
    def to_log(self) -> str:
        log_dict = asdict(self)
        
        log_dict['XY'] = round(self.XY, 2)
        log_dict['ZZ'] = round(self.ZZ, 2)
        log_dict['h'] = round(self.h, 2)
        log_dict['cutoff'] = round(self.cutoff, 3)
        
        # Filter out keys where the value is None
        filtered_log_dict = {key: log for key, log in log_dict.items() if log is not None}
        
        # Create log string
        return "_".join(
            f"{key}={log}" for key, log in filtered_log_dict.items()
        )

@dataclass
class Results:
    ground_MPS: list[npt.NDArray]
    ground_state_energy: float
    Neumann_entropy: float
    ground_energies: list[float] | npt.NDArray[np.float64] = field(default_factory=list)
    spin_z: list[float] | npt.NDArray[np.float64] = field(default_factory=list)
    spin_correlations: list[float] | npt.NDArray[np.float64] = field(default_factory=list)
    
    
    def to_array(self):
        return replace(
            self, **{field: np.array(getattr(self, field))
            if isinstance(getattr(self, field), list) else getattr(self, field)
            for field in asdict(self)}
        )
        
    def self_destruct(self):
        for field_name in self.__dataclass_fields__:
            field_value = getattr(self, field_name)
            if isinstance(field_value, list):
                field_value.clear()  # Clear list contents
            else:
                setattr(self, field_name, None)  # Set to None if not a list



In [ ]:
n_sites_lists = np.array([4096, 8192])
NKeeps = np.array([128])

XY_coupling = 1
magnetic_field = 0

ZZ_coupling_start = 0.8
ZZ_coupling_end = 1.4
delta_ZZ = 0.01

ZZ_couplings = bk.arange(ZZ_coupling_start, ZZ_coupling_end+delta_ZZ, delta_ZZ)

In [ ]:
for NKeep in NKeeps:
    for n_sites in n_sites_lists:
        for ZZ_coupling in ZZ_couplings:
            
            if verbose:
                print(f"L={n_sites} | ZZ={round(ZZ_coupling,2)}")
            
            Hamiltonian = XXZ_model(
                bk=bk, n_sites=n_sites, ZZ_coupling=ZZ_coupling,
                XY_coupling=XY_coupling, magnetic_field=magnetic_field,
            )
            
            ground_energies, ground_times, ground_MPS = DMRG(
                bk = bk,
                Hamiltonian = Hamiltonian,
                NKeep = NKeep,
                NSweep = NSweep,
                Krylov_bases = Krylov_bases,
                Lanczos_cutoff = Lanczos_cutoff,
                iterative_diag = True,
                two_site = two_site,
                verbose = verbose,
                tol = 1e-6,
            )
            
            param = Params(
                L = n_sites,
                XY = XY_coupling,
                ZZ = ZZ_coupling,
                h = magnetic_field,
                NKeep = NKeep,
                NSweep = NSweep,
                diag = True,
                Krylov = Krylov_bases,
                cutoff = Lanczos_cutoff,
                two = two_site
            )
            
            # dir_name = Path(f"data")
            dir_name = Path(f"{device}/critical_point")
            dir_name.mkdir(parents=True, exist_ok=True)
            
            filename = f"{param.to_log()}"
            ground_state_energy = ground_energies[-1]
            
            if verbose:
                print(f"computing Neumann entropy: ", end="")
                now = time.perf_counter()
            Neumann_entropy = get_Neumann_entropy_from_left_isometry(ground_MPS, bk=bk)
            if verbose:
                print(f"{round(time.perf_counter()-now,3)}s")
            
            result = Results(
                ground_MPS = [],
                ground_state_energy = ground_state_energy,
                ground_energies = ground_energies,
                Neumann_entropy = Neumann_entropy,
                spin_z = [],
                spin_correlations = [],
            )
            
            output: dict[str, Any] = {}
            output.update(asdict(param))
            output.update(asdict(result))
            
            with open(dir_name / f"{filename}.pkl", "wb") as file:
                pickle.dump(output, file)
        

L=4096 | ZZ=1.77
L=4096 | NKeep=128 | NSweep=20 | iterative=True | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=-2344.7987 | time=0js
iter=3 | energy=(-2344.852605774174+0j) | time=1192.0782458560134s
computing Neumann entropy: 193.02s
L=4096 | ZZ=1.78
L=4096 | NKeep=128 | NSweep=20 | iterative=True | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=-2352.5458 | time=0js
iter=2 | energy=(-2352.5996027463416+0j) | time=436.17064095297246s
computing Neumann entropy: 59.527s
L=4096 | ZZ=1.79
L=4096 | NKeep=128 | NSweep=20 | iterative=True | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=-2360.3127 | time=0js
iter=2 | energy=(-2360.3663083538627+0j) | time=599.1922498939966s
computing Neumann entropy: 50.349s
L=4096 | ZZ=1.8
L=4096 | NKeep=128 | NSweep=20 | iterative=True | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=-2368.0991 | time=0js
iter=2 | energy=(-2368.152628890787+0j) | time=379.39678256100